In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from functools import partial

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np

In [2]:
DEVICE = torch.device("cpu" if torch.cuda.device_count() < 1 else "cuda:0")
batch_size = 4
CLASSES = 2
DIR = os.getcwd()
EPOCHS = 5
N_TRAIN_EXAMPLES = batch_size * 30
N_VALID_EXAMPLES = batch_size * 10

print(f"Device: {DEVICE}\nBatch size: {batch_size}\nClasses: {CLASSES}\n\
Dir: {DIR}\nEpochs: {EPOCHS}\n\
Number of training examples: {N_TRAIN_EXAMPLES}\n\
Number of validation examples: {N_VALID_EXAMPLES}")

Device: cuda:0
Batch size: 4
Classes: 2
Dir: /home/bernardo/github/sex-age-estimation/backup-bia/patch-1
Epochs: 5
Number of training examples: 120
Number of validation examples: 40


In [3]:
model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

In [4]:
from torchvision import transforms as T
img_size = 224

transform = T.Compose([
                T.Resize((img_size,img_size)),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

In [5]:
from torch.utils.data import DataLoader
from configs import Inputs
from utils.augmentations import get_transforms
from utils.data import FullRadiographSexDataset

val_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                       fold_nums=Inputs().val_folds,
                                       transforms=get_transforms(Inputs(), subset=["train"]))

val_dataloader = DataLoader(val_dataset,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=0)

train_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                         fold_nums=Inputs().train_folds,
                                         transforms=get_transforms(Inputs(), subset=["train"]))

train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=0)

Using only horizontal flip augmentation.
Using only horizontal flip augmentation.


In [6]:
config = {
    "lr": tune.loguniform(1e-5, 1e-1),
    "optimizer_name": tune.choice(["Adam", "RMSprop", "SGD"])
}

In [7]:
def objective(config):

    # Gerar o modelo
    #model = define_model(trial).to(DEVICE)
    model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model.to(DEVICE)

    # Gerar optimizer
    optimizer_name = config['optimizer_name']
    lr = config["lr"]
    
    print("opt_name:", optimizer_name, "\nlr:", lr)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = train_dataloader, val_dataloader

    criterion = F.nll_loss

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if epoch_steps % 5 == 0:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
            running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(val_dataloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1
        
        accuracy= correct / total

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss.cpu(), accuracy=accuracy)

    return accuracy

In [ ]:
scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=EPOCHS,
        grace_period=1,
        reduction_factor=2)

reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(partial(objective),
                  resources_per_trial={"cpu": 2, "gpu": 1},
                  config=config,
                  num_samples=N_TRAIN_EXAMPLES,
                  scheduler=scheduler,
                  progress_reporter=reporter,
)

2022-12-27 14:21:01,152	INFO worker.py:1518 -- Started a local Ray instance.
2022-12-27 14:21:02,104	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


== Status ==
Current time: 2022-12-27 14:21:02 (running for 00:00:00.32)
Memory usage on this node: 2.8/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_14-21-02
Number of trials: 16/120 (15 PENDING, 1 RUNNING)
+-----------------------+----------+------------------+-------------+------------------+
| Trial name            | status   | loc              |          lr | optimizer_name   |
|-----------------------+----------+------------------+-------------+------------------|
| objective_b0fc9_00000 | RUNNING  | 172.17.0.2:11077 | 0.00283735  | RMSprop          |
| objective_b0fc9_00001 | PENDING  |                  | 2.76561e-05 | RMSprop          |
| objective_b0fc9_00002 | PENDING  |                  | 6.27052e-05 | RMSprop          |
| objective_b0fc9_00003 | PE

(raylet) [2022-12-27 14:21:11,042 E 10781 10800] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_14-20-59_184671_10713 is over 95% full, available space: 5481140224; capacity: 455568355328. Object creation will fail if spilling is required.


(func pid=11077) [1,    20] loss: -6.543
(func pid=11077) [1,    25] loss: -7.328
(func pid=11077) [1,    30] loss: -6.173
(func pid=11077) [1,    35] loss: -6.897
== Status ==
Current time: 2022-12-27 14:21:14 (running for 00:00:12.82)
Memory usage on this node: 4.5/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_14-21-02
Number of trials: 17/120 (16 PENDING, 1 RUNNING)
+-----------------------+----------+------------------+-------------+------------------+
| Trial name            | status   | loc              |          lr | optimizer_name   |
|-----------------------+----------+------------------+-------------+------------------|
| objective_b0fc9_00000 | RUNNING  | 172.17.0.2:11077 | 0.00283735  | RMSprop          |
| objective_b0fc9_00001 | PENDING  |      

(raylet) [2022-12-27 14:21:21,048 E 10781 10800] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_14-20-59_184671_10713 is over 95% full, available space: 5481132032; capacity: 455568355328. Object creation will fail if spilling is required.


...

(func pid=11510) [2,    15] loss: -1364.811
== Status ==
Current time: 2022-12-27 14:28:33 (running for 00:07:31.66)
Memory usage on this node: 4.5/15.6 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 4.000: 5956.771484375 | Iter 2.000: 2199.6865234375 | Iter 1.000: 493.7550354003906
Resources requested: 2.0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_14-21-02
Number of trials: 20/120 (16 PENDING, 1 RUNNING, 3 TERMINATED)
+-----------------------+------------+------------------+-------------+------------------+---------------------+------------+----------------------+
| Trial name            | status     | loc              |          lr | optimizer_name   | loss                |   accuracy |   training_iteration |
|-----------------------+------------+------------------+-------------+------------------+---------------------+------------+----------------------|
| objective_b0fc9_00003

(raylet) [2022-12-27 14:28:41,311 E 10781 10800] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_14-20-59_184671_10713 is over 95% full, available space: 5204742144; capacity: 455568355328. Object creation will fail if spilling is required.


(func pid=11510) [2,    50] loss: -722.736
(func pid=11510) [2,    55] loss: -652.063
== Status ==
Current time: 2022-12-27 14:28:43 (running for 00:07:41.68)
Memory usage on this node: 4.5/15.6 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 4.000: 5956.771484375 | Iter 2.000: 2199.6865234375 | Iter 1.000: 493.7550354003906
Resources requested: 2.0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_14-21-02
Number of trials: 20/120 (16 PENDING, 1 RUNNING, 3 TERMINATED)
+-----------------------+------------+------------------+-------------+------------------+---------------------+------------+----------------------+
| Trial name            | status     | loc              |          lr | optimizer_name   | loss                |   accuracy |   training_iteration |
|-----------------------+------------+------------------+-------------+------------------+---------------------+------------+-----

(raylet) [2022-12-27 14:28:51,317 E 10781 10800] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_14-20-59_184671_10713 is over 95% full, available space: 5204742144; capacity: 455568355328. Object creation will fail if spilling is required.


(func pid=11510) [2,    90] loss: -638.763
(func pid=11510) [2,    95] loss: -602.787
== Status ==
Current time: 2022-12-27 14:28:53 (running for 00:07:51.70)
Memory usage on this node: 4.5/15.6 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 4.000: 5956.771484375 | Iter 2.000: 2199.6865234375 | Iter 1.000: 493.7550354003906
Resources requested: 2.0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_14-21-02
Number of trials: 20/120 (16 PENDING, 1 RUNNING, 3 TERMINATED)
+-----------------------+------------+------------------+-------------+------------------+---------------------+------------+----------------------+
| Trial name            | status     | loc              |          lr | optimizer_name   | loss                |   accuracy |   training_iteration |
|-----------------------+------------+------------------+-------------+------------------+---------------------+------------+-----